In [7]:
import os, sys, glob
import pandas as pd
import numpy as np

import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import cloudpickle

def save_as_pkl( obj, fpath ):
    fwb = open( fpath, 'wb')
    tmp = fwb.write(cloudpickle.dumps(obj))
    return

In [4]:
# https://tech.unifa-e.com/entry/2018/09/20/183742
# > git clone https://github.com/davidsandberg/facenet.git

# add path for facenet model
from facenet.src import facenet

class FaceEmbedding(object):

    def __init__(self, model_path):
        facenet.load_model(model_path)
        
        self.input_image_size = 160
        self.sess = tf.Session()
        self.images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
        self.embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
        self.phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
        self.embedding_size = self.embeddings.get_shape()[1]
        
    def __del__(self):
        self.sess.close()
        
    def load_image(self, image_path, width, height, mode):
        image = Image.open(image_path)
        image = image.resize([width, height], Image.BILINEAR)
        return np.array(image.convert(mode))
        
    def face_embeddings(self, image_path):
        image = self.load_image(image_path, self.input_image_size, self.input_image_size, 'RGB')
        prewhitened = facenet.prewhiten(image)
        prewhitened = prewhitened.reshape(-1, prewhitened.shape[0], prewhitened.shape[1], prewhitened.shape[2])
        feed_dict = { self.images_placeholder: prewhitened, self.phase_train_placeholder: False }
        embeddings = self.sess.run(self.embeddings, feed_dict=feed_dict)
        return embeddings

In [ ]:
# read csv

df = pd.read_csv( '../../data/keyakizaka46_blog.csv' )
print(df.shape)
df.head()

In [6]:
# save facenet features

# load model
FACE_MEDEL_PATH = '../../model/20180408-102900/20180408-102900.pb'
face_embedding = FaceEmbedding(FACE_MEDEL_PATH)

save_dir = '../../data/images'

dirs = os.listdir( save_dir )
for author in dirs:
    if os.path.exists( '../../feature/image/facenet/X_facenet_%s.pkl' % (author) ): continue
    print( author, "Extract features..." )
    X = []
    y = []
    image_list  = []
    images = os.listdir( '%s/%s'% (save_dir,author) )
    for idx, imname in enumerate( images ):
        if idx % 100 == 0: print( idx )
        img_path = '%s/%s/%s'% (save_dir,author,imname)
        feature = face_embedding.face_embeddings( img_path )[0]
        X.append( feature.flatten() )
        y.append( author )
        image_list.append( img_path )
    
    # save feature
    X = np.array(X)
    save_as_pkl( X, '../../feature/image/facenet/X_facenet_%s.pkl' % (author) )
    save_as_pkl( y, '../../feature/image/facenet/y_facenet_%s.pkl' % (author) )
    save_as_pkl( image_list, '../../feature/image/facenet/imagelist_facenet_%s.pkl' % (author) )

Model filename: ../model/20180402-114759/20180402-114759.pb


In [ ]:
# sample

In [8]:
# ダウンロードした学習済みのモデルを解凍してパスを指定します
# FACE_MEDEL_PATH = '../../model/20180402-114759/20180402-114759.pb'
FACE_MEDEL_PATH = '../../model/20180408-102900/20180408-102900.pb'
face_embedding = FaceEmbedding(FACE_MEDEL_PATH)

In [10]:
# 顔画像のファイルリスト
faces_image_paths = glob.glob('./*.jpg')

ResourceExhaustedError: OOM when allocating tensor of shape [256] and type float
	 [[Node: InceptionResnetV1/Mixed_7a/Branch_2/Conv2d_1a_3x3/BatchNorm/beta/read/_87__cf__87 = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [256] values: -0.288294137 -0.197671518 -0.175553516...>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

In [ ]:
# 顔画像から特徴ベクトルを抽出
features = np.array([face_embedding.face_embeddings(f)[0] for f in faces_image_paths])
print(features.shape) # ==> (112, 128) 顔画像112枚が128次元表現になります